# 8. マルコフ連鎖モンテカルロ(MCMC)法とベイズ統計モデル

* 7章では観測できない不均質さを内包しているデータに対処するために、GLMMを導入した。
* 7章では個体差、場所差のどちらかのみ。
* 個体差、場所差の両方を同時に考慮した統計モデルが必要になることもある。
* ランダム効果の発生源が増えるとパラメータの推定が困難になる。
* そこで、このように複雑な統計モデルの当てはめで威力を発揮するのが、**マルコフ連鎖モンテカルロ法**(Markov chain Monte Carlo method,MCMC method)。
* MCMCアルゴリズムは、多変量の確率分布からの巧妙な乱数発生方法であり、統計モデルを観測データに当てはめるとMCMCサンプルが得られる。

## 8.1. 例題：種子の生存確率（個体差なし）

* 20個体から各8個の種子の生死を調べた。
* ヒストグラムは図8.1(B)
* 生存種子数$y_i$が二項分布に従うと仮定すると、
$$
p(y_i | q) = \binom{8}{y_i} q^{y_i} (1-q)^{8-y_i}
$$
* 尤度$L(q)$は20個体分の「データが得られる確率」の積なので
$$
L(q) = p({\bf{Y}}|q) = \prod_i p(y_i|q)
$$
* 両辺の対数を取ると
$$
\log L(q) = \sum_i \{ y_i \log q +(8-y_i) log(1-q)\} + \bf{C}
$$
* この対数尤度が最大になるのは、$\frac {d\log L(q)} {dq} = 0$となる$\hat{q}$。求めると、
$$
\hat{q} = \frac {Survival Num Of Seeds}{Survey Num Of Seeds} = \frac {73}{8\times 20} = 0.45625
$$
* 上記は解析的な、生存確率$\hat q$の求め方。次節では、解析的に求められない場合を検討する。

## 8.2. ふらふら試行錯誤による最尤推定

* 仮に最尤推定量$\hat q$が解析的に求められなくても、繰り返し試行錯誤によって対数尤度が高くなる$\hat q$を探し出す事ができる。
* ここでは効率が悪く、精度も良くない試行錯誤による最尤推定方法を紹介する。


1. 図8.2のグラフを元に$q$を飛び飛びの値にして離散化する。
2. ある$q$を選択する。
3. 選択した$q$からランダムに隣を選ぶ。
4. 対数尤度が今の$q$より高ければそちらに移動する。
5. 3,4を繰り返す
6. すると、$q$の初期値に関係なく、対数尤度の山を登りながら、$q$の値が最尤推定値の方向に変化していく。

## 8.3. MCMCアルゴリズムのひとつ：メトロポリス法

* MCMCアルゴリズムの説明に入る。
* メトロポリス法はMCMCアルゴリズムの中で最も簡単
* 8.2の最尤推定法の手順を少し修正すると、メトロポリス法になる。

1. パラメータ$q$の初期値を選ぶ
* $q$を増やすか減らすかランダムに決める
* $q^{New}$において尤度が大きくなる（当てはまりが良くなる)なら$q$の値を$q^{New}$に変更する。
* $q^{New}$で尤度が小さくなる（当てはまりが悪くなる）場合でも、確率$r=\frac {L(q^{New})}{L(q)}$で$q$の値を$q^{New}$に変化させる。

* 4の$r$の式を見ると、分子と分母の差が小さいほど$r=1$に近づき、移動しやすくなる。
* 一つのステップの中で、前の状態$q$にもとづいて新しい状態$q^{New}$を作り出しているので**マルコフ連鎖**
    * [wiki](https://ja.wikipedia.org/wiki/%E3%83%9E%E3%83%AB%E3%82%B3%E3%83%95%E9%80%A3%E9%8E%96)抜粋
    * マルコフ連鎖は、確率過程の一種であるマルコフ過程のうち、とりうる状態が離散的（有限または可算）なもの（離散状態マルコフ過程）をいう。
    * マルコフ連鎖は、未来の挙動が現在の値だけで決定され、過去の挙動と無関係である（マルコフ性）。
    * 遷移確率が過去の状態によらず、現在の状態のみによる系列である。
* 乱数を利用した計算アルゴリズムは**モンテカルロ法**

### 8.3.1. メトロポリス法でサンプリングしてみる
* メトロポリス法では$\log L(q)$の増大、その場での停滞だけでなく、減少するステップもある。
* 尤度が大きい（当てはまりが良い）所に到達するまで時間がかかる。
* 一旦対数尤度最大の地点に到達しても、そこには止まらずに$q$がでたらめに変化し続けているかのように見える。

Q. MCMCアルゴリズムの目的は？


A.
* ステップ数とともに変化するパラメーターの値の生成(サンプリング）
* 何か特定の値の探索ではない

---

* ステップ数が増えれば増えるほど、ヒストグラムの形状が、マルコフ連鎖の「定常分布」に近づいていく

### 8.3.2. マルコフ連鎖の定常分布

Q. 定常分布とは？

A.
* ある変数$q$のマルコフ連鎖が一定の条件を満たしている時に、そのマルコフ連鎖から発生する$q$の値が従う確率分布。

---

* 図8.8を見ると、ステップ数が増えるほど、サンプルされた$q$の分布は定常分布に近づく。
    * $q$の値をいい加減に決めて、さらに$q$の値が少しずつしか変化しないので定常分布に近づくのに時間がかかる。
    
* 異なる$q$の初期値から開始した複数のメトロポリス法の試行例が図8.9。
    * どのような初期値から開始しても、最終的には定常分布に従うように見える。
    
* 定常分布$p(q|\bf{Y})$を近似できるような$q$の標本集合を得るためには十分な数のMCMCサンプリングが必要。
    * ある$q$とそこから生成される新しい$q$の間に相関があるため。
* 効率の良いMCMCサンプリングを実現するには色々な方法がある。
    * メトリポリス法よりも「良い」MCMCアルゴリズム法を使う。
        * あるステップと次のステップでサンプルされた値の相関を低くするアルゴリズム(9.6.1ギブスサンプリング)
    * 初期状態を捨てる
        * いい加減に決めた初期値の影響を受けている。
    * 複数のMCMCサンプリングを比較する
        * 「定常分布への近づき具合」がわかりやすい。

### 8.3.3.この定常分布は何を表す分布なのか？

* 尤度$L(q)$に比例する確率分布
* 尤度$L(q)$に比例する離散化した$q$の確率分布とは、
$$
p(q|\bf{Y})= \frac {L(q)}{\sum_q L(q)}
$$


* 分母はすべての$L(q)$を足し合わせた値
    * すべての$q$について$p(q|\bf{Y})$を足し合わせると、$1$になる。

####  ここまでをまとめると

* ある観測データ$\bf {Y}$を説明するために、二項分布を部品とする統計モデルを作った。（図8.1)
* このモデルとメトロポリス法を使ってMCMCサンプリングした。（図8.8)
* すると、尤度に比例する$q$の確率分布$p(q|\bf{Y})$を推定できるようなサンプルが得られた(図8.11)

* このように推定された$p(q|\bf{Y})$は$q$の尤もらしさである尤度$L(q)$に比例しているので、あるデータ$\bf{Y}$に統計モデルを当てはめた時に$q$が取る値の確率分布と解釈しても良い。
* つまり、ここまでやってきたMCMCサンプリングとは、統計モデルの当てはめの一種だった。

## 8.4.  MCMCサンプリングとベイズ統計モデル

* MCMCサンプリングは統計モデルを観測データに当てはめる方法の一つ
* その結果として、与えられたデータとモデルの元でのパラメータ(生存確率)の確率分布が得られた。

* パラメータの確率分布と述べたが、これは簡単に使うことはできない。
    * 最尤推定法によるパラメータ推定は、統計学の枠組みの一つである頻度主義を前提
        * 頻度主義では、現象の背後にあるパラメータはある1個の「真の値」
        * それに対応する推定値も、データに基づいて決まる1個の数値
        * ばらつきを持った確率分布、確率変数ではない。
    * 考え方を変える。
* ベイズ統計学は、統計モデルのパラメータを確率分布として扱う枠組み。
    * ベイズ統計学で使う統計モデルでは、推定したいパラメータは確率分布として表現される。
    * この章のメトロポリス法による統計モデルの当てはめを、ベイズ統計学の枠組みで捉え直し、生存種子数の統計モデルをベイズ統計モデルとして見直してみる。

### ベイズ統計を例題に当てはめる

* ベイズ統計は、ベイズの公式の形式で推論を行う統計学。
* ベイズの公式をこの章の例題に当てはめると下記。
$$
p(q | {\bf Y}) = \frac {p( {\bf Y} |q) p(q)}{\sum_q p({\bf Y} |q) p(q)}
$$
* 左辺$p(q |{\bf Y}) $は**事後分布**。データ${\bf Y}$が得られた時に$q$が従う確率分布
* 右辺の分子の$p({\bf Y} |q)$は$q$の値が決まっている時にデータ$\bf {Y}$が観測される確率。この例題の場合、$p({\bf Y} |q)=L(q)$
* 右辺の分子の$p(q)$はデータ$\bf {Y}$がない時の$q$の確率分布。ベイズ統計モデルでは**事前分布**と呼ぶ。
    * 「データがない時の種子の生存確率$q$の事前分布」と言ってもよくわからないが、後回し。(9.3.事前分布)
* 右辺の分母は、すべての$q$について$p(q|{Y})$の和を取った時に1となるよう設定されているように見える。
    * また$\sum_q p({\bf Y}|q) p(q)$とは $p( {\bf Y})$のこと。$p({\bf Y})$は$q$の値が不明である時に$\bf {Y}$というデータが得られる確率。$q$の値によらない定数。
    
* つまりベイズ統計モデルとは下記の構造を持つ統計モデル。
    * 事後分布=(尤度x事前分布)/データが得られる確率
    * つまり、事後分布は尤度x事前分布に比例する。

### MCMCサンプリングとベイズ統計モデルを比較

* 8.3.3 で定常分布は尤度に比例する確率として、下記と表された。
$$
p(q|{\bf Y})= \frac {L(q)}{\sum_q L(q)}
$$

* ベイズの公式からは下記が得られた。
$$
p(q | {\bf Y}) = \frac {L(q) p(q)}{\sum_q L(q) p(q)}
$$
* 比較すると、事前分布$p(q)$が$q$の値によらずに$p(q)=$(定数) となっていると辻褄が合っているように見える。

* 実際のベイズ統計モデリングでは、モデル設計の段階で尤度・事前分布をきちんと指定し、それに整合するようにMCMCサンプリングを実施して、事後分布を推定する。

## 8.5. 補足説明

### 8.5.1. メトロポリス法と定常分布の関係

* メトロポリス法に従って、パラメータ$q$のサンプリングを行うと、定常分布$p(q|\bf Y)$になる理由について。

* 教科書参照

### 8.5.2. ベイズの定理

Q. ベイズの定理を数式で表すと？

A.
$$
p(q|{\bf Y}) = \frac {p({\bf Y}|q) p(q)}{\sum_q p({\bf Y} | q) p(q)}
$$

---

* ベイズの定理は、条件付き確率と同時確率の関係を整理したものに過ぎない
    * その関係とは、$p(A|B) p(B) = p(A,B)$
        * Bが成り立つ確率と、Bが成り立つ時Aが成り立つ確率(条件付き確率)の掛け算は、AとBが同時に成り立つ確率(同時確率)と等しい。

* （教科書とはちょっと説明の仕方が違いますが）
* 　$p({\bf Y},q)$ (または、$p(q,{\bf Y})$)は下記の2通りで表せる。
$$
p({\bf Y},q) = p(q|{\bf Y}) p({\bf Y})
$$
$$
p({\bf Y},q) = p({\bf Y}|q) p(q)
$$
* 合わせると、
$$
 p(q|{\bf Y}) p({\bf Y})=p({\bf Y}|q) p(q)
$$
* 下記のように変形できる。
$$
 p(q|{\bf Y})=\frac {p({\bf Y}|q) p(q)} {p({\bf Y})}
$$
* 上記はよく見かける「ベイズの公式」
* $ p(q|{\bf Y})$は事後分布、$p({\bf Y}|q)$は尤度に比例する確率、$p(q)$は$q$の事前分布。
* 右辺の分母の$p({\bf Y})$はデータを得られる確率であり、これは同時確率$p({\bf Y},q) $を全ての$q$について足し合わせたものなので、
$$
p({\bf Y}) = \sum_q p({\bf Y}|q)p(q)
$$
* 上記で書き換えると最初の式になり、
$$
p(q|{\bf Y}) = \frac {p({\bf Y}|q) p(q)}{\sum_q p({\bf Y} | q) p(q)}
$$

## 8.6. この章のまとめと参考文献
* 多数のパラメーターを推定する方法であるMCMC法、MCMCと対応の良さそうな統計モデルの一例として、ベイズ統計モデルを扱った。
* 最尤推定法は尤度最大になるパラメーターを探索する最適化である
* これに対し、MCMCアルゴリズムは定常分布からのランダムサンプリングが目的
    * この例題の場合、定常分布は尤度に比例する確率分布である
* 今扱っている統計モデルがベイズ統計モデルであるとすると、定常分布は事後分布であるとみなせる

### 非ベイズの統計モデルとベイズ統計モデルの違い

* 非ベイズはデータへの当てはまりの良さが尤度で評価されるので、その尤度を最大化するようにパラメーターを選んだ
* ベイズ統計モデルでは、尤度だけでなくパラメーターの事前分布も組み込まれているので、事後分布が当てはめの結果として得られる。